<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/LangChain_OpenAI_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Create a Moderation system with LangChain and OpenAI.</h2>
    <p>by <b>Pere Martra</b></p>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

# How To Create a Moderation System Using LangChain.

We are goin to create a Moderation System based in two Models. The first Model  reads the User comments and answer them.

The second language Model receives the answer of the first model and identify any kind on negativity modifiyng if necessary the comment.

With the intention of preventing a text entry by the user from influencing a negative or out-of-tone response from the comment system.

In [1]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.0
#!pip install -q openai==0.28.1
!pip install -q langchain-openai==0.0.2


In [2]:
!pip show langchain-openai

Name: langchain-openai
Version: 0.0.2
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core, numpy, openai, tiktoken
Required-by: 


## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.
* LLMChain: To create chains, where the prompts or the results can pass from one step to another inside the chain.

In [3]:
#PrompTemplate is a custom class that provides funcrionality to create prompts
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain




In [4]:
import torch
import os
import numpy as np

We need an OpenAI key to interac with the OpenAI API.

Here you can acces to your keys.
https://platform.openai.com/account/api-keys

OpenAI it's a pay service and you need a credit card to get a Key. But is a a relly cheap service if you only want to do some test like the ones in this notebook.

I'm using the gpt-3.5 as a moderator.


In [5]:
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [6]:
assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Create the template for the first model called **assistant**.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [7]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} social media post commenter, you will respond to the user, using the same sentiment than the user.
User:" {customer_request}"
Comment:
"""

In [8]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.  

In [9]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False,
)
#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the **.run** method of the chain, and pass the variables necessaries.

In our case: *customer_request* and *sentiment*.

In [10]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .run method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [11]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """This product is a piece of shit. I feel like an Idiot!"""


In [12]:
# Our assistatnt working in 'nice' mode.
response_data=create_dialog(customer_request, "nice")
print(f"assistant response: {response_data['assistant_response']}")

assistant response: I completely understand your frustration with this product. It's really disappointing when something doesn't meet our expectations. Hang in there, and hopefully things will improve for you.


The answer obtained is really polite. It dosn't need moderation.

In [13]:
#Our assistant running in rude mode.
response_data = create_dialog(customer_request, "rude")
print(f"assistant response: {response_data['assistant_response']}")

assistant response: Yeah, this product is absolute garbage. I can't believe I wasted my money on it. I feel like a complete moron for falling for their marketing tactics.


As you can see the answers we obtain are not polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.


In [14]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform to positive. Avoid any negatives words.
If it its nice, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
Edited comment:
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [15]:
#I'm going to use a more advanced LLM
moderator_llm = ChatOpenAI(model="gpt-4")

In [16]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [17]:
# To run our chain we use the .run() command
moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})

In [18]:
print(moderator_data['text'][97:])

, 'assistant_response': "Yes, this product didn't live up to my expectations. It's unfortunate that it didn't work out for me. I feel like I could have made a better choice."}


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.

In [19]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [20]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True,
)

In [21]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "nice", "customer_request": customer_request})



> Entering new SequentialChain chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform to positive. Avoid any negatives words.
If it its nice, you will let it remain as is and repeat it word for word.
Original comment: I completely understand your frustration with this product. It can be really disappointing when something doesn't live up to our expectations. Hang in there!
Edited comment:


> Finished chain.

> Finished chain.


"I completely understand your perspective on this product. It can be quite surprising when something doesn't quite match our expectations. Keep going!"

Lets use our Moderating System!

In [22]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "rude", "customer_request": customer_request})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform to positive. Avoid any negatives words.
If it its nice, you will let it remain as is and repeat it word for word.
Original comment: "Yeah, this product is a complete waste of money. I can't believe I fell for it either. Total disappointment!"
Edited comment:


> Finished chain.

> Finished chain.


'"Indeed, this product is an unexpected investment. I\'m surprised by the unique experience it provided. Quite the revelation!"'

Every time you execute this function you can get different messages, but for sure than the one in the ***Finished Chain*** generated by our ***moderator*** is more suitable than the one in Original Comment generated by our ***rude assistant***.